# CANTAL

On commence par installer 🤗 Transformers et 🤗 Datasets.

In [1]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 245kB 8.2MB/s 
     |████████████████████████████████| 2.5MB 41.6MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 245kB 54.7MB/s 
     |████████████████████████████████| 122kB 58.6MB/s 
     |████████████████████████████████| 901kB 41.0MB/s 
     |████████████████████████████████| 3.3MB 32.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16184 sha256=315501f7b7b6f09f8380b0c7dd306ad376ff7f3c85e6ce2501cc51036a000f7e
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: transformers 4.8.1 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


# Fine-tuner un modèle pour la classification de tokens

Nous pouvons fine-tuner un modèle de [🤗 Transformers](https://github.com/huggingface/transformers) pour la classification de tokens, i.e., pour attribuer à chaque token une etiquette.

Les tâches de classification de tokens les plus communes sont :

- NER (Named-entity recognition) Classification des entités (personne, organisation, lieu...).
- POS (Part-of-speech tagging) Étiquetage morpho-syntaxique des tokens (nom, verbe, adjectif...)

Nous allons entraîner le modèle de langue CamemBERT pour le NER sur le Corpus CLEF-HIPE

In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "camembert-base"
batch_size = 8

## Charger le dataset

Nous utiliserons la bibliothèque 🤗 Datasets pour charger les données et obtenir la métrique que nous devons utiliser pour l'évaluation. Cela peut être facilement fait avec les fonctions `load_dataset` et `load_metric`.

In [3]:
from datasets import load_dataset, load_metric

Ici nous utilisons le script `hipe.py` pour charger et preparer les données d'entraînement.

In [4]:
datasets = load_dataset('hipe.py', data_files={'train': 'hipe/train.conll', 'validation': 'hipe/dev.conll', 'test': 'hipe/test.conll'})

Using custom data configuration default-638374142bf104c4


Dataset presto downloaded and prepared to /root/.cache/huggingface/datasets/presto/default-638374142bf104c4/0.0.0/c8893ddc576a7be516281a116a9882b1e8aa07437c36683de1421adba9036c0f. Subsequent calls will reuse this data.


The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

L'objet `datasets` est une instance de [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict) qui contient les ensembles d'entraînement, de validation et de test.

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 4359
    })
    validation: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 1166
    })
    test: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 1013
    })
})

Pour accéder à une phrase, vous devez d'abord sélectionner un esemble, puis donner un index :

In [6]:
datasets["train"][0]

{'id': '0',
 'ner_tags': [9,
  9,
  9,
  4,
  0,
  9,
  9,
  1,
  11,
  11,
  11,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9],
 'tokens': ['NOUVELLES',
  'SUISSES',
  '—',
  'En',
  '1887',
  ',',
  'la',
  'Société',
  'suisse',
  'du',
  'Grutli',
  's',
  "'",
  'est',
  'accrue',
  'de',
  '40',
  'sections',
  ';',
  'l',
  "'",
  'association',
  'compte',
  'actuellement',
  '12,000',
  'membres',
  '.']}

Les étiquettes sont déjà codées sous forme d'identifiants entiers pour être facilement utilisables par notre modèle, mais la correspondance avec les catégories réelles est stockée dans les `features` de l'ensemble de données :

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['I-time',
 'B-org',
 'I-prod',
 'I-pers',
 'B-time',
 'I-loc',
 'B-loc',
 'B-comp',
 'B-prod',
 'O',
 'B-pers',
 'I-org']

## Prétraitement des données



Avant de pouvoir utiliser ces textes pour fine-tuner notre modèle, nous devons les prétraiter. Ceci est fait par un 🤗 Transformers Tokenizer qui (comme son nom l'indique) tokenisera les entrées et les mettra dans un format attendu par le modèle. Pour faire cela, nous instancions notre tokenizer avec la méthode `AutoTokenizer.from_pretrained`



In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Nous pouvons essayer notre tokenizer avec un phrase:

In [9]:
tokenizer("Bonjour, voici un phrase !")

{'input_ids': [5, 1285, 7, 1510, 23, 3572, 83, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Si, comme c'est le cas ici, vos entrées ont déjà été divisées en mots, vous devez transmettre la liste de mots à votre tokenzier avec l'argument `is_split_into_words=True` :

In [10]:
tokenizer(["Bonjour", ",", "voici", "une", "phrase", "découpée", "en", "mots", "!"], is_split_into_words=True)

{'input_ids': [5, 1285, 21, 7, 1510, 28, 3572, 13829, 35, 22, 883, 83, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Notez que les transformateurs sont souvent pré-entraînés avec des tokenizers de sous-mots, ce qui signifie que même si vos entrées ont déjà été divisées en mots, chacun de ces mots pourrait être à nouveau divisé par le tokenizer. Regardons un exemple de cela :

In [11]:
tokenized_input = tokenizer(["Et", "voici", "une", "phrase", "découpée", "conteanant", "un", "mot", "long", ":", "Anticonstitutionnellement", "!"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁Et', '▁voici', '▁une', '▁phrase', '▁découpé', 'e', '▁conte', 'an', 'ant', '▁un', '▁mot', '▁long', '▁:', '▁Anti', 'c', 'onstitutionnelle', 'ment', '▁!', '</s>']


Ici, les mots « decoupée », « contenant » et « Anticonstitutionnellement » ont été divisés en sous-mots.

Cela signifie que nous devons effectuer un certain traitement sur nos étiquettes car les identifiants d'entrée renvoyés par le tokenizer sont plus longs que les listes d'étiquettes contenues dans notre ensemble de données, d'abord parce que des tokens spéciaux peuvent être ajoutés (nous pouvons un `<s>` et un `</s>` ci-dessus), puis à cause de ces divisions possibles de mots en plusieurs sous-mots.

Voici la fonction qui prétraitera nos échantillions :

In [14]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Pour appliquer cette fonction sur toutes les phrases (ou paires de phrases) de notre jeu de données, nous utilisons simplement la méthode `map` de notre objet `dataset` que nous avons créé précédemment. Cela appliquera la fonction sur tous les éléments de toutes les ensembles de `dataset`, de sorte que nos données d'entraînement, de validation et de test seront prétraitées en une seule commande.

In [15]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## Fine-tutuner le modèle

Maintenant que nos données sont prêtes, nous pouvons télécharger le modèle pré-entraîné et le fine-tuner. Étant donné que toutes nos tâches concernent la classification des tokens, nous utilisons la classe `AutoModelForTokenClassification`. Comme avec le tokenizer, la méthode `from_pretrained` téléchargera et mettra en cache le modèle pour nous. La seule chose que nous devons spécifier est le nombre d'étiquettes pour notre tâche (que nous pouvons obtenir à partir des features, comme vu précédemment) :

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas

Pour instancier un `Trainer`, nous devrons définir trois autres choses. Le plus important est le [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), qui est une classe qui contient tous les attributs pour personnaliser l'entraînement. Il nécessite un nom de dossier, qui sera utilisé pour enregistrer les checkpoints du modèle. Tous les autres arguments sont facultatifs : 

In [17]:
args = TrainingArguments(
    f"hipe-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,
    weight_decay=0.01,
)

Ici, nous définissons l'évaluation à effectuer à la fin de chaque époque, ajustons le taux d'apprentissage, utilisons le `batch_size` défini en haut du cahier et personnalisons le nombre d'époques pour l'entraînement, ainsi que la diminution du poids.

Ensuite, nous aurons besoin d'un assembleur de données qui regroupera nos exemples traités tout en appliquant un _padding_ pour les rendre tous de la même taille. Il existe un assembleur de données pour cette tâche dans la bibliothèque Transformers, qui remplit non seulement les entrées, mais aussi les étiquettes :

In [18]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

La dernière chose à définir pour notre `Trainer` est de savoir comment calculer les métriques à partir des prédictions. Ici, nous allons charger la métrique [`seqeval`](https://github.com/chakki-works/seqeval) (qui est couramment utilisée pour évaluer les résultats sur l'ensemble de données CoNLL) via la bibliothèque Datasets.

In [19]:
metric = load_metric("seqeval")

Nous devrons donc faire un peu de post-traitement sur nos prédictions :
- sélectionner l'index prédit (avec le logit maximum) pour chaque token
- le convertir dans son étiquette (pers, loc, org, ...)
- ignorer tous les tokens spéciaux 

La fonction suivante effectue tout ce post-traitement sur le résultat de `Trainer.evaluate` (qui est un tuple nommé contenant des prédictions et des étiquettes) avant d'appliquer la métrique

In [20]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Notez que nous laissons tomber la précision/rappel/f1 calculée pour chaque catégorie et que nous nous concentrons uniquement sur la précision/rappel/f1/exactitude globale.

Ensuite, nous avons juste besoin de transmettre tout cela avec nos ensembles de données au `Trainer` :

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Nous pouvons maintenant fine-tuner notre modèle en appelant simplement la méthode `train` :

In [22]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running training *****
  Num examples = 4359
  Num Epochs = 7
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3815


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.799400,0.385228,0.624639,0.658667,0.641202,0.949866
2,0.315300,0.237444,0.769140,0.742476,0.755573,0.963716
3,0.208200,0.204024,0.777949,0.771429,0.774675,0.965486
4,0.153900,0.162830,0.791233,0.811429,0.801204,0.971067
5,0.117100,0.145984,0.802996,0.816762,0.809821,0.972098
6,0.099400,0.144581,0.800815,0.824000,0.812242,0.971425
7,0.080100,0.144224,0.800074,0.823238,0.811491,0.971560


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Evaluation *****
  Num examples = 1166
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in

TrainOutput(global_step=3815, training_loss=0.23926543789484866, metrics={'train_runtime': 1120.7485, 'train_samples_per_second': 27.226, 'train_steps_per_second': 3.404, 'total_flos': 3188255914550952.0, 'train_loss': 0.23926543789484866, 'epoch': 7.0})

La méthode `evaluate` vous permet d'évaluer à nouveau sur le jeu de données d'évaluation ou sur un autre jeu de données :

In [23]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Evaluation *****
  Num examples = 1166
  Batch size = 8


{'epoch': 7.0,
 'eval_accuracy': 0.9715598386373824,
 'eval_f1': 0.8114907998497934,
 'eval_loss': 0.14422394335269928,
 'eval_precision': 0.8000740466493891,
 'eval_recall': 0.8232380952380952,
 'eval_runtime': 10.4067,
 'eval_samples_per_second': 112.043,
 'eval_steps_per_second': 14.029}

Pour obtenir la précision/rappel/f1 calculée pour chaque catégorie maintenant que nous avons terminé l'apprentissage, nous pouvons appliquer la même fonction que précédemment sur le résultat de la méthode `predict` :

In [27]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags, id.
***** Running Prediction *****
  Num examples = 1166
  Batch size = 8


{'loc': {'f1': 0.8710341985990936,
  'number': 1208,
  'precision': 0.867104183757178,
  'recall': 0.875},
 'org': {'f1': 0.6159695817490495,
  'number': 244,
  'precision': 0.574468085106383,
  'recall': 0.6639344262295082},
 'overall_accuracy': 0.9715598386373824,
 'overall_f1': 0.8114907998497934,
 'overall_precision': 0.8000740466493891,
 'overall_recall': 0.8232380952380952,
 'pers': {'f1': 0.8121730860675225,
  'number': 1028,
  'precision': 0.7944186046511628,
  'recall': 0.830739299610895},
 'prod': {'f1': 0.5625000000000001,
  'number': 72,
  'precision': 0.6428571428571429,
  'recall': 0.5},
 'time': {'f1': 0.732394366197183,
  'number': 73,
  'precision': 0.7536231884057971,
  'recall': 0.7123287671232876}}